In [6]:
import os
import requests
import requests

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from time import sleep


from locations import *
from parsing import *

# LOCATIONS SCRAPING

In [7]:
# Base

base_url = 'https://www.pisos.com/'

venta_url = f'{base_url}venta/'

In [8]:
# Sacamos todas las paginas filtradas por provincias

response = requests.get(base_url)

soup = BeautifulSoup(response.text, "html.parser")

provincias_str = soup.find('div', class_ = 'home-container').find('div', class_ = 'selectBox').find_all('ul')[-1].text

provincias = [x.lower() for x in provincias_str.strip().split('\n')]

endpoints = replaceWithUnderscore(provincias)

endpoints = [venta_url + endpoint for endpoint in endpoints]

with open('locations.txt', 'a+') as file:
    for endpoint in endpoints:
        file.writelines(endpoint + '\n')

# SINGLE PAGE SCRAPING

In [9]:
def scrape(url):

    browser = webdriver.Chrome()
    browser.get(url)
    browser.maximize_window()

    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    )

    element.click() # Accept cookies

    for i in range(30):

        element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, f'(//*[@id]/div[2]/div[1]/div[2]/a)[{i + 1}]'))
        )

        element.click() # Click on the first post

        # <SCROLLING>
        while True:
            is_at_bottom = browser.execute_script("return window.scrollY + window.innerHeight >= document.body.scrollHeight")
            browser.execute_script("window.scroll({ top: document.body.scrollHeight, behavior: 'smooth' });")
            if is_at_bottom:
                break
            sleep(0.2)
        # </SCROLLING>
        
        html_content = browser.page_source
        location = 'test'

        file_path = f'{os.getcwd()}/html_content/{location}_{i}.html'
        try:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(html_content)
        except Exception as e:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(repr(e))

        go_back_element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="GoBackLink"]'))
        )

        go_back_element.click() # Click on back arrow

    browser.quit()

    

In [10]:
test_url = endpoints[4]

data = scrape(test_url)

# TESTING

In [58]:
url = 'https://www.pisos.com/viviendas/barcelona/'

def scanMap(url):
    response = requests.get(url)

    print(f'URL: {url} | STATUS {response.status_code}')

    soup = BeautifulSoup(response.text, 'html.parser')

    items = soup.select('div.zoneList a.item')

    endpoints = {}
    for item in items:
        endpoint = item['href']
        n_results = item.find('span', class_ = 'total').text

        # <INT>
        if len(n_results) != 0:
            try:
                n_results = n_results[1:-1]
                n_results = ''.join(n_results.split('.'))
                n_results = int(n_results)
            except:
                print(f'FAIL CASTING TO INTEGER {endpoint}')
        else:
            print(f'n_results EMPTY {endpoint}')
        #</INT>

        # <RECURSSION>
        if n_results > 3000:
            endpoints[endpoint] = scanMap(base_url[:-1] + endpoint)
        # </RECURSSION>

        else: endpoints[endpoint] = n_results

    return endpoints

endpoints = scanMap(url)

URL: https://www.pisos.com/viviendas/barcelona/ | STATUS 200
URL: https://www.pisos.com/viviendas/baix_llobregat/ | STATUS 200
URL: https://www.pisos.com/viviendas/barcelones/ | STATUS 200
URL: https://www.pisos.com/viviendas/barcelona_capital/ | STATUS 200
URL: https://www.pisos.com/venta/pisos-barcelona_capital/ | STATUS 200
URL: https://www.pisos.com/viviendas/maresme/ | STATUS 200
URL: https://www.pisos.com/viviendas/valles_occidental/ | STATUS 200
URL: https://www.pisos.com/viviendas/valles_occidental_sud/ | STATUS 200


In [68]:
def parseRegions(endpoints):
    def extract(endpoints):
        array = []
        for key, value in endpoints.items():
            if isinstance(value, int):
                array.append(key)
                continue

            data = extract(value)
            array.extend(data)

        return array
    
    endpoints = extract(endpoints)

    array = []
    for endpoint in endpoints:
        data = endpoint
        if '/venta/pisos-' in endpoint:
            data = endpoint.replace('/venta/pisos-', '/viviendas/')
        array.append(data)
    return array


In [71]:
parseRegions(endpoints = endpoints)

['/viviendas/alt_penedes/',
 '/viviendas/vilafranca_del_penedes/',
 '/viviendas/sant_sadurni_anoia/',
 '/viviendas/castellet_i_la_gornal/',
 '/viviendas/torrelles_de_foix/',
 '/viviendas/gelida/',
 '/viviendas/anoia/',
 '/viviendas/piera/',
 '/viviendas/igualada/',
 '/viviendas/masquefa/',
 '/viviendas/anoia_els_hostalets_de_pierola/',
 '/viviendas/vilanova_del_cami/',
 '/viviendas/bages/',
 '/viviendas/manresa/',
 '/viviendas/sant_vicenc_de_castellet/',
 '/viviendas/castellbell_i_el_vilar/',
 '/viviendas/sallent/',
 '/viviendas/navarcles/',
 '/viviendas/baix_llobregat_nord/',
 '/viviendas/esparreguera/',
 '/viviendas/olesa_de_montserrat/',
 '/viviendas/martorell/',
 '/viviendas/sant_andreu_de_la_barca/',
 '/viviendas/abrera/',
 '/viviendas/baix_llobregat_sud_est/',
 '/viviendas/sant_boi_de_llobregat/',
 '/viviendas/esplugues_de_llobregat/',
 '/viviendas/cornella_de_llobregat/',
 '/viviendas/viladecans/',
 '/viviendas/sant_vicenc_dels_horts/',
 '/viviendas/baix_llobregat_sud_oest/',
 '